In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn import model_selection

import sklearn
import sklearn.naive_bayes
import sklearn.tree
import seaborn
import matplotlib

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv ("/kaggle/input/titanic/train.csv", index_col="PassengerId") 
df_train.head()


In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
#some features do not matter when predicting if a passanger survived or not

df_train.pop("Ticket")
df_train.pop("Name")
df_train.pop("Pclass")
df_train.pop("Embarked")
df_train.pop("Cabin")

df_train.head()

In [ ]:
#we now apply a label enconder for non-numeric values 

lbl_enc = preprocessing.LabelEncoder()
df_train["Sex"] = lbl_enc.fit_transform(df_train["Sex"])


df_train.head()


In [ ]:
#we saw before that the feature "Age" has some null values. We could delete this values
#however it was decided to change Null values by the Median of the data in that column


median_age=df_train["Age"].median() #median=1

df_train['Age'] = df_train['Age'].replace(np.nan, median_age)

In [ ]:
#is possible to see that more women survied than man even though we changed NaN values to 1
#that is due possibly by that fact that in a disaster women and children are rescued first
#although, there can be a erro associated with the probability due to the fact that I change the NaN values by their median


#we can also see that most people were in their late 20's
#that being said



woman_n=df_train.loc[df_train["Sex"]==0].shape[0]
man_n=df_train.loc[df_train["Sex"]==1].shape[0]


fig, axes = matplotlib.pyplot.subplots(2,2, figsize = (20,10))


df_lived = df_train.loc[df_train["Survived"]==1]
df_dead = df_train.loc[df_train["Survived"]==0]

seaborn.barplot(x=df_train["Sex"], y=df_train["Survived"],ax=axes[0,0])
plot1=seaborn.histplot(df_train, x="Age", kde=True, bins=50,hue="Sex",stat="count", ax=axes[0,1])
plot2=seaborn.histplot(df_lived, x="Age",bins=50, stat="count", kde=True, hue="Sex", ax=axes[1,0])
plot3=seaborn.histplot(df_dead, x="Age",bins=50, stat="count", kde=True, hue="Sex", ax=axes[1,1])


plot1.set_title("Distribution of People by Age/Sex")
plot2.set_title ("Distribution of Alive People by Age/Sex")
plot3.set_title ("Distribution of Dead People by Age/Sex")



print("Number of Women in Dataframe: {}".format(woman_n))
print("Number of Man in Dataframe: {}".format(man_n))


matplotlib.pyplot.show()

In [ ]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split (df_train[["Sex", "Age", "SibSp", "Parch", "Fare"]], df_train[["Survived"]], test_size=0.25)


print("Logistic Regression\n")


model=sklearn.linear_model.LogisticRegression(max_iter=200)

model.fit(X_train, Y_train)

Y_predict = model.predict(X_train)
print(sklearn.metrics.classification_report (Y_train, Y_predict))



print("Naive Bayes with Gaussian Distribution\n")

model3 = sklearn.naive_bayes.GaussianNB()

model3.fit(X_train, Y_train)

Y_predict3 = model3.predict(X_train)
print(sklearn.metrics.classification_report (Y_train, Y_predict3))



print("Decision Tree max Depth 10\n")

model2 = sklearn.tree.DecisionTreeClassifier(max_depth=8)

model2.fit(X_train, Y_train)

Y_predict2 = model2.predict(X_train)
print(sklearn.metrics.classification_report (Y_train, Y_predict2))




 


In [ ]:
sklearn.tree.plot_tree(model2)

In [ ]:
df_test = pd.read_csv ("/kaggle/input/titanic/test.csv", index_col="PassengerId") 
df_test.head()

In [ ]:
df_test.pop("Ticket")
df_test.pop("Name")
df_test.pop("Pclass")
df_test.pop("Embarked")
df_test.pop("Cabin")

df_test.head()

In [ ]:
print(df_test.isna().sum())

In [ ]:

df_test["Sex"].replace("female", 0, inplace=True)
df_test["Sex"].replace("male", 1, inplace=True)


female_df = df_test.loc[(df_test["Sex"] == 0) & (df_test ["Sex"] != np.nan)]
male_df = df_test.loc[(df_test["Sex"] == 1) & (df_test ["Sex"] != np.nan)]

median_value_female = female_df["Age"].median()
median_value_male = male_df["Age"].median()

print(median_value_female)
print(median_value_male)


df_test.loc[df_test["Sex"]==0,"Age"] = df_test.loc[df_test["Sex"]==0,"Age"].replace(np.nan,median_value_female)
df_test.loc[df_test["Sex"]==1,"Age"] = df_test.loc[df_test["Sex"]==1,"Age"].replace(np.nan,median_value_male)

print(df_test.isna().sum())

In [ ]:
median_fare = df_test["Fare"].median()

print(median_fare)

df_test["Fare"]=df_test["Fare"].replace(np.nan,median_fare)
print(df_test.isna().sum())

In [ ]:
X_test_final = df_test[["Sex","Age","SibSp","Parch","Fare"]]
#Y_test_final = df_test["Survived"]

Y_predict_final = model2.predict(X_test_final)

#print(sklearn.metrics.classification_report (Y_test_final, Y_predict_final))


In [ ]:
df_submission = pd.DataFrame (X_test_final["Fare"])
df_submission["Fare"] = Y_predict_final

df_submission.columns=["Survived"]
print(df_submission.shape)
df_submission.head(10)

In [ ]:
df_submission.to_csv("Final_Csv.csv")